In [1]:
import os
from pathlib import Path
import sys

if "__project_dir__" not in globals():
    __project_dir__ = Path.cwd().parents[1].resolve()

sys.path.append(__project_dir__)
os.chdir(__project_dir__)

In [2]:
import pandas as pd

In [5]:
data_dir = Path("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data")

### Load data

In [6]:
raw = pd.read_csv(data_dir/"raw"/"gdb"/"gdb_20230606.csv")

### Focus on beta chains

In [12]:
beta_raw = raw.drop(columns=["TRAV", "CDR3A", "TRAJ"])
beta_raw = beta_raw[beta_raw.CDR3B.notna()]

In [17]:
beta_raw.head()

,TRBV,CDR3B,TRBJ,MHCA,MHCB,Epitope,studies,dbs
0,TRBV4-1*01,CASSQDMKGGSFTGELFF,TRBJ2-2*01,HLA-B*42:01,B2M,FPRPWLHGL,PMID:25911754,VDJdb
1,TRBV7-2*01,CASSLFSGGDQPQHF,TRBJ1-5*01,HLA-B*42:01,B2M,FPRPWLHGL,PMID:25911754,VDJdb
2,TRBV7-2*01,CASSLWAGGPETQYF,TRBJ2-5*01,HLA-B*42:01,B2M,FPRPWLHGL,PMID:25911754,VDJdb
3,TRBV7-9*01,CASSLGPTVQGNYGYTF,TRBJ1-2*01,HLA-B*42:01,B2M,HPKVSSEVHI,PMID:25911754,VDJdb
4,TRBV5-5*01,CASSLEGTSGPQETQYF,TRBJ2-5*01,HLA-B*42:01,B2M,TPQDLNTML,PMID:25911754,VDJdb


In [20]:
beta_raw_vdjdb_subset = beta_raw[beta_raw.dbs.str.contains("VDJdb")]

In [21]:
beta_raw_vdjdb_subset

,TRBV,CDR3B,TRBJ,MHCA,MHCB,Epitope,studies,dbs
0,TRBV4-1*01,CASSQDMKGGSFTGELFF,TRBJ2-2*01,HLA-B*42:01,B2M,FPRPWLHGL,PMID:25911754,VDJdb
1,TRBV7-2*01,CASSLFSGGDQPQHF,TRBJ1-5*01,HLA-B*42:01,B2M,FPRPWLHGL,PMID:25911754,VDJdb
2,TRBV7-2*01,CASSLWAGGPETQYF,TRBJ2-5*01,HLA-B*42:01,B2M,FPRPWLHGL,PMID:25911754,VDJdb
3,TRBV7-9*01,CASSLGPTVQGNYGYTF,TRBJ1-2*01,HLA-B*42:01,B2M,HPKVSSEVHI,PMID:25911754,VDJdb
4,TRBV5-5*01,CASSLEGTSGPQETQYF,TRBJ2-5*01,HLA-B*42:01,B2M,TPQDLNTML,PMID:25911754,VDJdb
...,...,...,...,...,...,...,...,...
101939,TRBV13-3*01,CASSDWGNTGQLYF,TRBJ2-2*01,MH2-D1,NaN,SQLLNAKYL,PMID:29150238,"VDJdb, IEDB"
101940,TRBV13-3*01,CASSDDWGSSQNTLYF,TRBJ2-4*01,MH2-D1,NaN,SQLLNAKYL,PMID:29150238,"VDJdb, IEDB"
101941,TRBV13-3*01,CASSDWGATGQLYF,TRBJ2-2*01,MH2-D1,NaN,SQLLNAKYL,PMID:29150238,"VDJdb, IEDB"
101942,TRBV13-3*01,CASSNWGNQDTQYF,TRBJ2-5*01,MH2-D1,NaN,SQLLNAKYL,PMID:29150238,"VDJdb, IEDB"
